# VesselMNIST3D Data

In [1]:
# Install libAUC and medMNIST
!pip install libauc==1.2.0
!pip install medmnist
!pip install tensorboardX
!pip install acsconv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=5f41d8b7194e1fae773d3c80798ec6ea45620b77e8fe2709db158e8387d08566
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import medmnist
from medmnist import INFO, Evaluator
import os
import random
import torchvision
from torchvision import datasets, models
from torch.utils.data import DataLoader
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.metrics import auc_roc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from acsconv.converters import Conv3dConverter
from libauc.models import resnet18 as ResNet18

The ``converters`` are currently experimental. It may not support operations including (but not limited to) Functions in ``torch.nn.functional`` that involved data dimension


# Importing the dataset

In [3]:
data_flag = 'vesselmnist3d'
download = True
as_rgb = True
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])

# Parameters

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
BATCH_SIZE = 64
imratio = 0.1
total_epochs = 101 #50 75 90
#decay_epochs = [50, 75]
shape_transform = True
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Loading and Transformation

In [13]:
# Loading and Transforming the data
class Transform3D:
    def __init__(self, mul=None):
        self.mul = mul
    def __call__(self, voxel):
        if self.mul == '0.5':
            voxel = voxel * 0.5
        elif self.mul == 'random':
            voxel = voxel * np.random.uniform()
        return voxel.astype(np.float32)

train_transform = Transform3D(mul='random') if shape_transform else Transform3D()
eval_transform = Transform3D(mul='0.5') if shape_transform else Transform3D()

train_dataset = DataClass(split='train', transform=train_transform, download=download, as_rgb=as_rgb)
train_dataset_at_eval = DataClass(split='train', transform=eval_transform, download=download, as_rgb=as_rgb)
val_dataset = DataClass(split='val', transform=eval_transform, download=download, as_rgb=as_rgb)
test_dataset = DataClass(split='test', transform=eval_transform, download=download, as_rgb=as_rgb)

train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset_at_eval, batch_size=2*BATCH_SIZE, shuffle=False)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

Using downloaded and verified file: /root/.medmnist/vesselmnist3d.npz
Using downloaded and verified file: /root/.medmnist/vesselmnist3d.npz
Using downloaded and verified file: /root/.medmnist/vesselmnist3d.npz
Using downloaded and verified file: /root/.medmnist/vesselmnist3d.npz


In [14]:
train_dataset

Dataset VesselMNIST3D (vesselmnist3d)
    Number of datapoints: 1335
    Root location: /root/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'vessel', '1': 'aneurysm'}
    Number of samples: {'train': 1335, 'val': 192, 'test': 382}
    Description: The VesselMNIST3D is based on an open-access 3D intracranial aneurysm dataset, IntrA, containing 103 3D models (meshes) of entire brain vessels collected by reconstructing MRA images. 1,694 healthy vessel segments and 215 aneurysm segments are generated automatically from the complete models. We fix the non-watertight mesh with PyMeshFix and voxelize the watertight mesh with trimesh into 28×28×28 voxels. We split the source dataset with a ratio of 7:1:2 into training, validation and test set.
    License: CC BY 4.0

# Training

In [15]:
# Defining the ResNet-18 3D model
model = ResNet18(pretrained=False)
model = model.cuda()
model.conv1 = nn.Conv2d(3, 64, kernel_size=3,stride=1, padding=1, bias=False)
model = Conv3dConverter(model)

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 2)
)

# LibAuc loss function and optimizer
loss_fn = AUCMLoss()
optimizer = PESG(model, loss_fn=loss_fn, momentum=0.7, margin=1.0, epoch_decay=2e-3, lr=0.1, weight_decay=1e-4) #epoch decay=0.05, 2e-3

best_val_auc = 0
best_test_auc = 0

for epoch in range(total_epochs):
    train_loss = []
    model = model.cuda()
    model.train()
    for data, targets in train_loader:
        data, targets = data.cuda(), targets.cuda()
        y_pred = model(data)
        loss = loss_fn(y_pred, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())

    model.eval()

    val_pred_list = []
    val_true_list = []
    for val_data, val_targets in val_loader:
         val_data = val_data.cuda()
         val_pred = model(val_data)
         val_pred_list.append(val_pred.cpu().detach().numpy())
         val_true_list.append(val_targets.numpy())
    val_true = np.concatenate(val_true_list)
    val_pred = np.concatenate(val_pred_list)
    val_auc =  auc_roc_score(val_true, val_pred)

    print("epoch: %s, val_auc: %.4f, lr: %.4f"%(epoch,val_auc[0],optimizer.lr ))    

    # Retain the best optimized model so we can use that to test
    if val_auc[0] > best_val_auc:
        best_val_auc = val_auc[0]
        final_model=model


epoch: 0, val_auc: 0.6067, lr: 0.1000
epoch: 1, val_auc: 0.4853, lr: 0.1000
epoch: 2, val_auc: 0.7594, lr: 0.1000
epoch: 3, val_auc: 0.6187, lr: 0.1000
epoch: 4, val_auc: 0.6684, lr: 0.1000
epoch: 5, val_auc: 0.6906, lr: 0.1000
epoch: 6, val_auc: 0.7877, lr: 0.1000
epoch: 7, val_auc: 0.7679, lr: 0.1000
epoch: 8, val_auc: 0.5126, lr: 0.1000
epoch: 9, val_auc: 0.7091, lr: 0.1000
epoch: 10, val_auc: 0.7754, lr: 0.1000
epoch: 11, val_auc: 0.8393, lr: 0.1000
epoch: 12, val_auc: 0.8452, lr: 0.1000
epoch: 13, val_auc: 0.8567, lr: 0.1000
epoch: 14, val_auc: 0.8471, lr: 0.1000
epoch: 15, val_auc: 0.8746, lr: 0.1000
epoch: 16, val_auc: 0.8388, lr: 0.1000
epoch: 17, val_auc: 0.8257, lr: 0.1000
epoch: 18, val_auc: 0.8890, lr: 0.1000
epoch: 19, val_auc: 0.7735, lr: 0.1000
epoch: 20, val_auc: 0.8195, lr: 0.1000
epoch: 21, val_auc: 0.8505, lr: 0.1000
epoch: 22, val_auc: 0.8003, lr: 0.1000
epoch: 23, val_auc: 0.7706, lr: 0.1000
epoch: 24, val_auc: 0.8107, lr: 0.1000
epoch: 25, val_auc: 0.8345, lr: 0.1

In [16]:
# Saving the final model
state = {
    'net': final_model.state_dict(),
}

torch.save(state, 'final_model_Vessel.pth')

# Testing

In [17]:
test_pred_list = []
test_true_list = [] 
for test_data, test_targets in test_loader:
    test_data  = test_data.cuda()
    test_pred = final_model(test_data)
    test_pred_list.append(test_pred.cpu().detach().numpy())
    test_true_list.append(test_targets.numpy())
test_true = np.concatenate(test_true_list)
test_pred = np.concatenate(test_pred_list)
test_auc =  auc_roc_score(test_true, test_pred) 

test_pred_binary = (test_pred > 0.5).astype(int)
#test_accuracy = accuracy_score(test_true, test_pred_binary)   
print(test_auc[0])  


0.8686286615901763


# Evaluation

In [18]:
# As given in Demo Eval file on canvas
def evaluate(net, test_loader):
    # Testing AUC
    score_list = list()
    label_list = list()
    for tmp_data, tmp_label in test_loader:
        # tmp_data, tmp_label, tmp_idx = data
        tmp_data, tmp_label = tmp_data.cuda(), tmp_label.cuda()      
        tmp_score = net(tmp_data).detach().clone().cpu()
        score_list.append(tmp_score)
        label_list.append(tmp_label.cpu())
    test_label = torch.cat(label_list)
    test_score = torch.cat(score_list)
                   
    test_auc = auc_roc_score(test_label, test_score)                   
    #print("Test: %.4f"%test_auc, flush=True)
    print("AUC: ",test_auc[0])

evaluate(final_model, test_loader)

AUC:  0.8686286615901763
